In [1]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm_notebook
import pandas as pd
from keras.utils import np_utils
from sklearn.cluster import KMeans
import csv
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10

In [3]:
def GetGSCFeatureData():
    gscSamePair = pd.read_csv('GSC-Dataset/GSC-Features-Data/same_pairs.csv') 
    gscDiffPair = pd.read_csv('GSC-Dataset/GSC-Features-Data/diffn_pairs.csv')
    #Reading human observed feature data csv 
    gscFeatureData = pd.read_csv('GSC-Dataset/GSC-Features-Data/GSC-Features.csv')
    #print(humanFeatureData.img_id)
    #print(humanSamePair.img_id_A, humanSamePair.img_id_B) 

    sameFeatureMerge=pd.merge(gscSamePair, gscFeatureData,  how='inner', left_on=['img_id_A'], right_on = ['img_id'], suffixes= ['_A','_A'])
    featureListSame=pd.merge(sameFeatureMerge, gscFeatureData, how='inner', left_on=['img_id_B'], right_on = ['img_id'], suffixes= ['_A','_B'])
    #print(featureListSame)

    diffFeatureMerge=pd.merge(gscDiffPair, gscFeatureData,  how='inner', left_on=['img_id_A'], right_on = ['img_id'], suffixes= ['_A','_A'])
    featureListDiff=pd.merge(diffFeatureMerge, gscFeatureData, how='inner', left_on=['img_id_B'], right_on = ['img_id'], suffixes= ['_A','_B'])
    return sameFeatureMerge, featureListSame, diffFeatureMerge, featureListDiff
    #print(featureListDiff.iloc[2])

In [4]:
sameFeatureMerge, featureListSame, diffFeatureMerge, featureListDiff = GetGSCFeatureData()
#Getting the array vector of Target values from 'same_pairs.csv'.
def ProcessData(featureListSame,featureListDiff):
    t1 = []
    t1_target=[]
    t2_target=[]
    columns =[2]
    columnnext=range(4,4+512)
    columnrest=range(5+512,5+512*2)
    #columns.extend(columnnext)
    #columns.extend(columnrest)
    #columns.extend(range(4,4+512)).extend(range(5+512,5+512*2))
    t1 = np.array(featureListSame.iloc[:,columnnext].values)
    t2 = np.array(featureListDiff.iloc[0:featureListSame.shape[0],columnrest].values)
    t1=np.subtract(t1,t2)
    t1=np.insert(t1, 4, values=columns, axis=1)
    np.random.shuffle(t1)
    print(t1)
    return t1
t1 = ProcessData(featureListSame,featureListDiff)
print(t1.shape)

[[-1 -1  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 1  1  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [-1 -1  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
(71531, 513)


# Getting data for the dataset

In [5]:
#Getting the matrix of total data
def GenerateRawData(t1):    
    dataMatrix = [] 
    
    dataMatrix = np.transpose(t1)     
    print ("Data Matrix Shape.."+ str(dataMatrix.shape))
    return dataMatrix

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(rawData.shape[0]*0.01*TrainingPercent))
    print("RawData"+str(rawData.shape))
    d2 = rawData[0:T_len,:]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2


#Generating the list of validation target values which is 10% of total length
#List of size 10% of the total length will be obtained.
def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(rawData.shape[0]*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount:V_End-1]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

GenerateRawData(t1)

def GenerateValData(rawData, ValPercent, TrainingCount): 
    #ceil() returns ceiling value of x i.e the smallest integer not less than x.
    valSize = int(math.ceil(rawData.shape[0]*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[TrainingCount:V_End-1,:]
    print("Validation Data Matrix"+str(dataMatrix.shape))  
    return dataMatrix

Data Matrix Shape..(513, 71531)


# Fetch and Prepare Dataset

In [6]:
RawData = np.array(t1[:,1:])
RawTarget = np.array(t1[:,0:1])
print (RawData.shape)
print (RawTarget.shape)

(71531, 512)
(71531, 1)


# Prepare Training Data

In [7]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
TrainingData= np.transpose(TrainingData)
print(TrainingTarget.shape)
print(TrainingData.shape)

RawData(71531, 512)
(57225, 1)
(512, 57225)


# Prepare Validation Data

In [8]:
#We have used np.array for ValDataAct because it is of list type and we want
# it to be of array type
#print(len(TrainingTarget))
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
ValData = np.transpose(ValData)
print(ValDataAct.shape)
print(ValData.shape)

Validation Data Matrix(7153, 512)
(7153, 1)
(512, 7153)


# Prepare Test Data

In [9]:
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
TestData = np.transpose(TestData)
print(ValDataAct.shape)
print(ValData.shape)

Validation Data Matrix(7153, 512)
(7153, 1)
(512, 7153)


# Sigmoid activation

In [10]:
def predict(features, weights):
    print("Feature shape :"+ str(features.shape))
    #print("Weights shape :"+ str(weights.shape))
    z = np.dot(features, weights)
    return sigmoid(z)

In [11]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

# Cost function

In [12]:
def cost_function(features, labels, weights):
    '''
    Using Mean Absolute Error

    Features:(100,3)
    Labels: (100,1)
    Weights:(3,1)
    Returns 1D matrix of predictions
    Cost = ( log(predictions) + (1-labels)*log(1-predictions) ) / len(labels)
    '''
    observations = len(labels)

    predictions = predict(features, weights)

    #Take the error when label=1
    class1_cost = -labels*np.log(predictions)

    #Take the error when label=0
    class2_cost = (1-labels)*np.log(1-predictions)

    #Take the sum of both costs
    cost = class1_cost - class2_cost

    #Take the average cost
    cost = cost.sum()/observations
    print("Cost: "+str(cost))

    return cost

In [13]:
def update_weights(features, labels, weights, lr):
    '''
    Vectorized Gradient Descent

    Features:(200, 3)
    Labels: (200, 1)
    Weights:(3, 1)
    '''
    N = len(features)

    #1 - Get Predictions
    predictions = predict(features, weights)

    #2 Transpose features from (200, 3) to (3, 200)
    # So we can multiply w the (200,1)  cost matrix.
    # Returns a (3,1) matrix holding 3 partial derivatives --
    # one for each feature -- representing the aggregate
    # slope of the cost function across all observations
    gradient = np.dot(features.T,  predictions - labels)

    #3 Take the average cost derivative for each feature
    gradient /= N

    #4 - Multiply the gradient by our learning rate
    gradient *= lr

    #5 - Subtract from our weights to minimize cost
    weights -= gradient

    return weights

In [14]:
def train(features, labels, weights, lr, iters):
    cost_history = []

    for i in range(iters):
        weights = update_weights(features, labels, weights, lr)

        #Calculate error for auditing purposes
        cost = cost_function(features, labels, weights)
        cost_history.append(cost)

        # Log Progress
        if i % 1000 == 0:
            print ("iter: "+str(i) + " cost: "+str(cost))

    return weights, cost_history

In [15]:
def accuracy_model(predicted_labels, actual_labels):
    count=0
    print("predicted_labels :"+ str(predicted_labels.shape))
    print("actual_labels :"+ str(actual_labels.shape))
    for i in range(len(predicted_labels)):
        if np.around(predicted_labels[i])==actual_labels[i]:
            count=count+1;
    accuracy= (count/len(predicted_labels))*100
    print("Accuracy: "+ str(accuracy))
    return accuracy

In [ ]:
#Training model for different hyperparameters RawData, RawTarget, Weights, learning rate, epochs
weights,cost=train(RawData,RawTarget,np.zeros((RawData.shape[1],1)),0.005,1000)
Cost_function=cost_function(RawData,RawTarget,0)
predictedVal=predict(RawData,weights)
print("predictedVal :"+str(predictedVal.shape)) 
Model_accuracy=accuracy_model(predict(RawData,weights),RawTarget)
#print("Cost function :"+str(Cost_function))

#In order to plot the graph
plt.plot([Cost_function,Model_accuracy])
plt.annotate('Cost_function', (0,Cost_function), (0,Cost_function),
            arrowprops=dict(facecolor='red', shrink=0.05),
            )
plt.annotate('Model_accuracy', (1,Model_accuracy), (1,Model_accuracy),
            arrowprops=dict(facecolor='green', shrink=0.05),
            )
plt.grid()
plt.show()

Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6874631387470496
iter: 0 cost: 0.6874631387470496
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6818395499318913
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6762756986034395
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6707708721551654
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.665324361207071
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6599354599113136
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6546034662416617
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6493276822670394
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6441074144094578
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6389419736866404
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6338306759396843
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.6287728420461

Feature shape :(71531, 512)
Cost: 0.32546279627898916
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3232278787037234
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3210092913597055
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3188068416230419
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3166203396383639
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3144495982744636
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.31229443308063687
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3101546622437241
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3080301065458421
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3059205893227994
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.303825936423187
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.3017459761681354
Feature shape :(71531, 512)
Feature shape :(71531, 512)

Feature shape :(71531, 512)
Cost: 0.15864767808129773
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.1574152811632782
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.15618859638177746
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.1549675723560249
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.1537521583021737
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.15254230402509555
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.15133795991030355
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.1501390769160016
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.14894560656525674
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.14775750093829498
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.14657471266491529
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.145397194917022
Feature shape :(71531, 512)
Feature shape :(71531,

Feature shape :(71531, 512)
Cost: 0.0563778817964872
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.05553683811082903
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.05469835330620275
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.053862409644455496
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.05302898955434781
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.05219807562964692
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.05136965062724662
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.050543697465310906
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.049720199221441726
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.04889913913087185
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.04808050058467973
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: 0.04726426712802917
Feature shape :(71531, 512)
Feature shape 

Feature shape :(71531, 512)
Cost: -0.016739072816018274
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.01739237743326541
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.018044295170737912
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.01869483361390627
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.019344000289626736
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.01999180266670742
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.020638248156467604
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.021283344113292035
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.021927097835177267
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.022569516564273102
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.023210607487417705
Feature shape :(71531, 512)
Feature shape :(71531, 512)
Cost: -0.023850377736666466
Feature shape :(71531, 